In [1]:
#Before using spark, need to install and set up spark, Hadoop and Jdk locally on your laptop


from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("etl") \
    .config("spark.master", "local") \
    .config("spark.jars", "F:/UniPrj/Big Data Project/postgresql-42.2.16.jar") \
    .getOrCreate()

#Load Amazon Data into Spark DataFrame
#1 load product segment - Personal Care Segment --> download dataset tsv file
file_path = "F:/UniPrj/Big Data Project/Datasets/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv"
df = spark.read.option("encoding", "UTF-8").csv(file_path, sep="\t", header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

In [2]:
#Create DataFrame - Perform Preliminary Cleaning

#1 Check the schema, print row & column count
df.printSchema()
print((df.count(), len(df.columns)))

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)

(85981, 15)


In [3]:
#2 Drop the round 1 columns
columns_to_drop = ['marketplace', 'product_parent', 'vine', 'review_headline', 'review_body', 'review_date']
df_dropped = df.drop(*columns_to_drop)
df_dropped.show()

+-----------+--------------+----------+--------------------+--------------------+-----------+-------------+-----------+-----------------+
|customer_id|     review_id|product_id|       product_title|    product_category|star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+--------------+----------+--------------------+--------------------+-----------+-------------+-----------+-----------------+
|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|                Y|
|   18125776|R3QWMLJHIW6P37|B0000537JQ|     Ezy Dose Weekly|Personal_Care_App...|          5|            0|          0|                Y|
|   19917519|R14Z1VR1N0Z9G6|B00HXXO332|Pulse Oximeter, B...|Personal_Care_App...|          5|            1|          1|                Y|
|   18277171| R25ZRJL0GH0U0|B00EOB0JA2|SE Tools Tool Kit...|Personal_Care_App...|          2|            0|          0|                Y|
|    2593270|R3837KYH7AZNIY|B00OC2

In [4]:

#3 Filter Step 1:
df_filtered = df_dropped.filter(df_dropped.verified_purchase == 'Y')
df_filtered.show
print((df_filtered.count(), len(df_filtered.columns)))

(63312, 9)


In [5]:
#4 drop filtered verified_purchase column
columns_to_drop = ['verified_purchase']
df_dropped_2 = df_filtered.drop(*columns_to_drop)
df_dropped_2.show()

+-----------+--------------+----------+--------------------+--------------------+-----------+-------------+-----------+
|customer_id|     review_id|product_id|       product_title|    product_category|star_rating|helpful_votes|total_votes|
+-----------+--------------+----------+--------------------+--------------------+-----------+-------------+-----------+
|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|
|   18125776|R3QWMLJHIW6P37|B0000537JQ|     Ezy Dose Weekly|Personal_Care_App...|          5|            0|          0|
|   19917519|R14Z1VR1N0Z9G6|B00HXXO332|Pulse Oximeter, B...|Personal_Care_App...|          5|            1|          1|
|   18277171| R25ZRJL0GH0U0|B00EOB0JA2|SE Tools Tool Kit...|Personal_Care_App...|          2|            0|          0|
|    2593270|R3837KYH7AZNIY|B00OC2O1UC|doTERRA HD Clear ...|Personal_Care_App...|          4|            0|          1|
|    2592955|R2MN0QYCY6EVIV|B00HES9CMS|V

In [6]:
#Create Analysis-Specific DFs/Tables
#Perform Analysis-Specific Transforms
#Segmentation Analysis DF

# 1 Create Segmentation DF by Droppig Addtional Columns
segmentation_cols_drop = ['review_id', 'product_id', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
segmentation_dropped_df = df_dropped_2.drop(*segmentation_cols_drop)
segmentation_dropped_df.show()

+-----------+--------------------+
|customer_id|    product_category|
+-----------+--------------------+
|   32114233|Personal_Care_App...|
|   18125776|Personal_Care_App...|
|   19917519|Personal_Care_App...|
|   18277171|Personal_Care_App...|
|    2593270|Personal_Care_App...|
|    2592955|Personal_Care_App...|
|   15168265|Personal_Care_App...|
|   13761624|Personal_Care_App...|
|   29615023|Personal_Care_App...|
|   47893062|Personal_Care_App...|
|    2582596|Personal_Care_App...|
|   21969415|Personal_Care_App...|
|   43153609|Personal_Care_App...|
|   17782951|Personal_Care_App...|
|   13710264|Personal_Care_App...|
|   30720884|Personal_Care_App...|
|   35206254|Personal_Care_App...|
|   10261156|Personal_Care_App...|
|   41933766|Personal_Care_App...|
|   28020969|Personal_Care_App...|
+-----------+--------------------+
only showing top 20 rows



In [7]:
#Note: must change Category Label name in withColumnRenamed('count(product_category)', 'name')
#2 Segmentation GroupBy
#2a GroupBy customer_id
#2b Count product_category and rename count columns as Segment Name
segment_df = segmentation_dropped_df.groupby("customer_id")\
.agg({'product_category':'count'}).withColumnRenamed('count(product_category)', 'personal_care_appliances')
#2c Check results
segment_df.show()

+-----------+------------------------+
|customer_id|personal_care_appliances|
+-----------+------------------------+
|   11160821|                       1|
|    1099354|                       1|
|   31581262|                       1|
|   27596904|                       1|
|    2405598|                       1|
|   11229098|                       1|
|   46000147|                       1|
|   15234640|                       1|
|   28937218|                       1|
|   27954895|                       1|
|   31689770|                       1|
|   30117841|                       1|
|   38591899|                       1|
|   24557051|                       1|
|    2780908|                       1|
|   11096259|                       1|
|    2416824|                       1|
|   33939357|                       1|
|     501949|                       1|
|   15676483|                       1|
+-----------+------------------------+
only showing top 20 rows



In [8]:
#3 Check segment_df Schema and Row Count
segment_df.printSchema()
print(segment_df.count())

root
 |-- customer_id: integer (nullable = true)
 |-- personal_care_appliances: long (nullable = false)

61769


In [9]:
#Note: Column Name in df.sort('name' ...) must align with Column name from step 2
#4 Filter for Top n Results
#4a Declare number of rows to filter by (100,000)
row_count = 100000
#4a Sort by Segment Desc and limit to row_count
filtered_segment_df = segment_df.sort('personal_care_appliances', ascending=False).limit(row_count)
#4b Check Results
filtered_segment_df.show()
print(filtered_segment_df.count())

+-----------+------------------------+
|customer_id|personal_care_appliances|
+-----------+------------------------+
|    3949232|                       8|
|    6660431|                       7|
|    1597288|                       5|
|   52082728|                       5|
|   13096644|                       5|
|   30882451|                       5|
|   16856119|                       5|
|   52392440|                       4|
|   18280728|                       4|
|     768135|                       4|
|   48401768|                       4|
|   16155270|                       4|
|   32399423|                       4|
|   43642145|                       4|
|   38193696|                       4|
|   51500774|                       4|
|   44415965|                       4|
|   11408281|                       4|
|   44441036|                       4|
|   45365967|                       4|
+-----------+------------------------+
only showing top 20 rows

61769


In [10]:
#Segmentation ETL Complete - Add Database Export Code

# Configure settings for RDS
mode = "append"
jdbc_url = 'jdbc:postgresql://localhost:5432/AmazonProducts'
config = {"user":'postgres', 
          "password": 'root', 
          "driver":"org.postgresql.Driver"}

In [11]:
#Note: table name in table='name_segment' must align with table name in Postgres


# Write segment table to Postgres/RDS
# 11 secs
filtered_segment_df.write.jdbc(url=jdbc_url, table='personal_care_appliances_segment', mode=mode, properties=config)
     

In [12]:
# Run Queries in Postgres to Confirm Segment ETL
# Check Row Count of Segment Table - SELECT COUNT(*) FROM personal_care_appliances_segment;

# Check 10 Rows of Segment Table - SELECT * FROM personal_care_appliances_segment LIMIT(10);


# Upon Confirmation of Above Checks Run This Cell
print('Segment ETL Successful')

Segment ETL Successful


In [13]:
# Apriori Analysis DF
# Enter Apriori Transfomations Below - Use df_dropped_2 as Start Point

# 1 Create Apriori DF by First Dropping Addtional Columns
apriori_cols_drop = ['product_category', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
apriori_dropped_df = df_dropped_2.drop(*apriori_cols_drop)
apriori_dropped_df.show()

+-----------+--------------+----------+
|customer_id|     review_id|product_id|
+-----------+--------------+----------+
|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|
|   18125776|R3QWMLJHIW6P37|B0000537JQ|
|   19917519|R14Z1VR1N0Z9G6|B00HXXO332|
|   18277171| R25ZRJL0GH0U0|B00EOB0JA2|
|    2593270|R3837KYH7AZNIY|B00OC2O1UC|
|    2592955|R2MN0QYCY6EVIV|B00HES9CMS|
|   15168265|R3AN2UJ1D42ZS0|B0016BFR4G|
|   13761624|R3U29ZLUWEEK4M|B00K504UUG|
|   29615023| RRRDOEJZD1Y22|B00P6TUO5G|
|   47893062|R2KR5ZEAS859DK|B0006VJ6TO|
|    2582596| RR7PGQY763IHF|B00H9L7VIW|
|   21969415| RN37YYZBP9BMP|B00P1JNZUW|
|   43153609| R1UXGB70OR5TW|B00N5HD340|
|   17782951|R1OC5ZNXSGO7P1|B0007DHMVK|
|   13710264|R1W4ZN8BPVNWK3|B0002JG2NI|
|   30720884|R2KE33CNN5X2B0|B00OYRW4UE|
|   35206254|R1A26EDSO0U5N6|B00598WT0C|
|   10261156|R3L81GOHVS3E9P|B0006VJ6TO|
|   41933766|R24V6VDHS7JMQV|B0006Z0NY2|
|   28020969|  RBA56WVCDD5M|B00H9L7VIW|
+-----------+--------------+----------+
only showing top 20 rows



In [14]:
#Prelim Apriori ETL Complete - Add Database Export Code

# Configure settings for RDS
mode = "append"
jdbc_url = 'jdbc:postgresql://localhost:5432/AmazonProducts'
config = {"user":'postgres', 
          "password": 'root', 
          "driver":"org.postgresql.Driver"}

In [15]:
#Note: table name in table='segment_apriori' must align with table name in Postgres


# Write segment_apriori table to RDS
# 5 mins
apriori_dropped_df.write.jdbc(url=jdbc_url, table='personal_care_appliances_apriori', mode=mode, properties=config)
     

In [16]:
# Run Queries in Postgres to Confirm Apriori ETL
# Check Row Count of Apriori Table - SELECT COUNT(*) FROM personal_care_appliances_apriori;

# Check 10 Rows of Apriori Table - SELECT * FROM personal_care_appliances_apriori LIMIT(10);


# Upon Confirmation of Above Checks Run This Cell
print('Apriori ETL Successful')

Apriori ETL Successful
